In [45]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import random
import itertools

In [46]:
name = 'FD001'
seq_len = 30

# Data Loading



In [47]:




def load_data(name):
    data_root = "data/units/"
    label_root = "data/new_labels/"
    lis = os.listdir(data_root)
    data_list = [i for i in lis if i in name]
    data, label, padding = [], [], []
    seq_len = 30
    Rc = 130



    for n in data_list:
        
        raw = np.loadtxt(data_root+n)[:,2:]
        
        lbl = np.loadtxt(label_root+n)/Rc
        
        l = len(lbl)
        if l<seq_len:
            raise RuntimeError("seq_len {} is too big for file '{}' with length {}".format(seq_len, n, l))
        raw, lbl = torch.tensor(raw, dtype=torch.float), torch.tensor(lbl, dtype=torch.float)
        lbl_pad_0 = [torch.ones([seq_len-i-1]) for i in range(seq_len-1)] 
        
        data_pad_0 = [torch.zeros([seq_len-i-1,24]) for i in range(seq_len-1)]
    
        lbl_pad_1 = [torch.zeros([i+1]) for i in range(seq_len-1)] 
        data_pad_1 = [torch.zeros([i+1,24]) for i in range(seq_len-1)]
        
        data += [torch.cat([data_pad_0[i],raw[:i+1]],0) for i in range(seq_len-1)] 
        data += [raw[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
        data += [torch.cat([raw[l-seq_len+i+1:], data_pad_1[i]],0) for i in range(seq_len-1)]
        label += [torch.cat([lbl_pad_0[i],lbl[:i+1]],0) for i in range(seq_len-1)] 
        label += [lbl[i-seq_len+1:i+1] for i in range(seq_len-1, l)]
        label += [torch.cat([lbl[l-seq_len+i+1:], lbl_pad_1[i]],0) for i in range(seq_len-1)]
        padding += [torch.cat([torch.ones(seq_len-i-1), torch.zeros(i+1)],0) for i in range(seq_len-1)]   # 1 for ingore
        padding += [torch.zeros(seq_len) for i in range(seq_len-1, l)]
        padding += [torch.cat([torch.zeros(seq_len-i-1), torch.ones(i+1)],0) for i in range(seq_len-1)]
    return data, label, padding


In [48]:
train = np.loadtxt("save/"+name+"/train"+name+".txt", dtype=str).tolist()
valid = np.loadtxt("save/"+name+"/valid"+name+".txt", dtype=str).tolist()
test = np.loadtxt("save/"+name+"/test"+name+".txt", dtype=str).tolist()

train_data, train_label, train_padding = load_data(train)
valid_data, valid_label, valid_padding = load_data(valid)
test_data, test_label, test_padding = load_data(test)


# Model

In [ ]:
model = transformer(max_len=seq_len)
model.to('mps')
Loss = nn.MSELoss()
Loss.to('mps')
opt = torch.optim.Adam(model.parameters(), lr=0.001)



In [ ]:
def train(data, label, padding, model, Loss, opt):
    min = 999
    for e in range(10):
        model.train()
        random.shuffle(data)
        total_loss = 0
        iter = iter(DataLoader(data, batch_size=32, shuffle=True))
        counter = 0
        for _ in range(len(iter)):
            train_data, train_label, train_padding = next(iter)
            train_data, train_label, train_padding = train_data.to('mps'), train_label.to('mps'), train_padding.to('mps')
            feature, out = model(train_data, train_padding)
            out.squeeze_(2)
            loss = Loss(out, train_label)
            total_loss += loss
            counter += 1
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        rmse = validate()
        print("Epoch: {}, Loss: {}, RMSE: {}".format(e, total_loss/counter, rmse))
        if (rmse < min):
            min = rmse
            torch.save(model.state_dict(), 'save/transformer.pth')
        return min
    
            
            
            
        
def validate():
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for i in range(len(valid_data)):
            feature, out = model(valid_data[i].unsqueeze(0).to('mps'), valid_padding[i].unsqueeze(0).to('mps'))
            out.squeeze_(2)
            loss = Loss(out, valid_label[i].unsqueeze(0).to('mps'))
            total_loss += loss

In [ ]:

def train():
    minn = 999
    for e in range(epochs):
        al, tot = 0, 0
        net.train()
        random.shuffle(source_list)
        random.shuffle(target_list)
        source_iter, target_iter = iter(source_list), iter(target_list)
        loss2_sum, loss1_sum = 0, 0
        bkb_sum, out_sum = 0, 0
        cnt = 0
        s_iter = iter(DataLoader(s_data, batch_size=args.batch_size, shuffle=True))
        t_iter = iter(DataLoader(t_data, batch_size=args.batch_size, shuffle=True))
        l = min(len(s_iter), len(t_iter))
        for _ in range(l):
            s_d, t_d = next(s_iter), next(t_iter)
            s_input, s_lb, s_msk = s_d[0], s_d[1], s_d[2]
            t_input, t_msk = t_d[0], t_d[2]
            s_input, s_lb, s_msk = s_input.cuda(), s_lb.cuda(), s_msk.cuda()
            t_input, t_msk = t_input.cuda(), t_msk.cuda()
            s_features, s_out = net(s_input, s_msk)
            t_features, t_out = net(t_input, t_msk) # [bts, seq_len, feature_num]
            s_out.squeeze_(2)
            t_out.squeeze_(2)
            loss1 = Loss(s_out, s_lb)
            loss1_sum += loss1
            cnt += 1
            if args.type == 1 or args.type == 0:
                if args.type == 1:
                    s_domain = D2(s_features)
                    t_domain = D2(t_features)
                else:
                    s_domain = D1(s_out)
                    t_domain = D1(t_out)
                loss2 = advLoss(s_domain.squeeze(1), t_domain.squeeze(1), 'cuda')
                loss2_sum += loss2
                loss = loss1 + a*loss2
            elif args.type == 2:
                s_domain_bkb = D2(s_features)
                t_domain_bkb = D2(t_features)
                s_domain_out = D1(s_out)
                t_domain_out = D1(t_out)
                if e>=5:
                    fea_loss = advLoss(s_domain_bkb.squeeze(1), t_domain_bkb.squeeze(1), 'cuda')
                    out_loss = advLoss(s_domain_out.squeeze(1), t_domain_out.squeeze(1), 'cuda')
                    bkb_sum += fea_loss
                    out_sum += out_loss
                    loss = loss1 + a*fea_loss + b*out_loss
                else:
                    loss = loss1
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(itertools.chain(net.parameters(), D1.parameters(), D2.parameters()), 2)
            opt.step()    

        rmse = validate()
        if args.type == 2:
            print("{}/{}| loss1={:.5f}, fea_loss={:.5f}, out_loss={:.5f}, rmse={:.5f}".\
                format(e, args.epoch, loss1_sum/cnt, bkb_sum/cnt, out_sum/cnt, rmse))
        else:    
            print("{}/{}| 1={:.5f}, 2={:.5f}, rmse={:.5f}".format(e, args.epoch, loss1, loss2_sum/cnt, rmse))
        if rmse<minn:
            minn = rmse
            print("min={}".format(minn))
            if args.type == 1:
                torch.save(net.state_dict(), "save/final/dann_"+source[-1]+target[-1]+".pth")
            elif args.type == 0:
                torch.save(net.state_dict(), "save/final/out_"+source[-1]+target[-1]+".pth")
            elif args.type == 2 :
                #torch.save(net.state_dict(), "save/final/both_"+source[-1]+target[-1]+".pth")
                torch.save(net.state_dict(), "online/"+source[-1]+target[-1]+"_net.pth")
                torch.save(D1.state_dict(), "online/"+source[-1]+target[-1]+"_D1.pth")
                torch.save(D2.state_dict(), "online/"+source[-1]+target[-1]+"_D2.pth")
        
        if args.scheduler:
            sch.step()

    return minn